In [21]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

df2 = pd.read_csv('huh.csv')

Как устроен этот блокнот: вначале проводится исследование нескольких моделей для данных с отброшенными категориальными столбцами. Затем все то же повторяется для данных с категориальными столбцами, уже без комментариев. В конце присутствует сводная таблица эффективности моделей для обоих случаев, небольшой вывод и параметры наиболее успешных моделей

Попробуем вместо label encoding просто отбросить категориальные данные

In [22]:
from sklearn.preprocessing import LabelEncoder

df = df2.select_dtypes(['number'])
df.head(5)

,Unnamed: 0,hour,day,month,price,distance,surge_multiplier,temperature,precipIntensity,precipProbability,...,apparentTemperatureLow,dewPoint,pressure,windBearing,cloudCover,uvIndex,ozone,moonPhase,precipIntensityMax,temperatureMin
0,0,9,16,12,5.0,0.44,1.0,42.34,0.0000,0.0,...,27.39,32.70,1021.98,57,0.72,0,303.8,0.30,0.1276,39.89
1,1,2,27,11,11.0,0.44,1.0,43.58,0.1299,1.0,...,36.20,41.83,1003.97,90,1.00,0,291.1,0.64,0.1300,40.49
2,2,1,28,11,7.0,0.44,1.0,38.33,0.0000,0.0,...,29.11,31.10,992.28,240,0.03,0,315.7,0.68,0.1064,35.36
3,3,4,30,11,26.0,0.44,1.0,34.38,0.0000,0.0,...,26.20,26.64,1013.73,310,0.00,0,291.1,0.75,0.0000,34.67
4,4,3,29,11,9.0,0.44,1.0,37.44,0.0000,0.0,...,30.29,28.61,998.36,303,0.44,0,347.7,0.72,0.0001,33.10


Вновь обучим модель линейной регрессии и dummy-регрессор. В прошлом эксперименте лучше показал себя dummy-регрессор со стратегией взятия среднего значения, используем его

Кроме того, каждый регрессор загрузим в соответствующий ему пайплайн с рескейлером

In [23]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

X_to_split = df.drop(['price','surge_multiplier'], inplace=False, axis=1)
Y_to_split = df['price']

X_train,  X_test, Y_train, Y_test = train_test_split(X_to_split, Y_to_split, random_state=100)

dummy1 = DummyRegressor(strategy='mean')
lr = LinearRegression()

pipelineDummy = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", dummy1),
    ]
)

pipelineLinear = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", lr),
    ]
)


pipelineDummy.fit(X_train, Y_train)
pipelineLinear.fit(X_train, Y_train)


Pipeline(steps=[('scaler', StandardScaler()), ('clf', LinearRegression())])

In [24]:
from sklearn.metrics import mean_squared_error

y_preds_dummy1 = pipelineDummy.predict(X_test)
print(f'MSE Dummy1 : {mean_squared_error(Y_test, y_preds_dummy1, squared=True)}')
y_preds_lr = pipelineLinear.predict(X_test)
print(f'MSE Linear : {mean_squared_error(Y_test, y_preds_lr, squared=True)}')

MSE Dummy1 : 86.90642906214326
MSE Linear : 76.52616961606286


Попробуем метод регрессии, основанный на методе N-ближайших соседей

In [25]:
from sklearn.neighbors import KNeighborsRegressor

nr = KNeighborsRegressor()

pipelineNR = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", nr),
    ]
)

parametersNR = {
    'clf__n_neighbors': [5,3,7,10],
    'clf__weights': ['uniform', 'distance']
}

grid_searchNR = GridSearchCV(
    estimator=pipelineNR,
    param_grid=parametersNR,
    n_jobs=-1,
    verbose=1,
)

grid_searchNR.fit(X_train, Y_train)

best_parametersNR = grid_searchNR.best_estimator_.get_params()
for param_name in sorted(parametersNR.keys()):
    print(f"{param_name}: {best_parametersNR[param_name]}")


y_preds_NR_opt = grid_searchNR.predict(X_test)

print(f'MSE Neighbors : {mean_squared_error(Y_test, y_preds_NR_opt, squared=True)}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
clf__n_neighbors: 10
clf__weights: uniform
MSE Neighbors : 85.53137763271972


Почему-то полученный результат (85.5) хуже линейной регрессии и почти равен результату глупого регрессора. Возможно, это связано с тем, что были отброшены категориальные данные. Попробуем еще два метода регрессии, регрессор, основанный на гауссовских процессах и регрессор, основаный на решающих деревьях

Вначале регрессор, основанный на решающих деревьях

In [26]:
from sklearn import tree

pipelineTree = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", tree.DecisionTreeRegressor()),
    ]
)

parametersTree  = {
    'clf__max_depth': [2,3,5,7,None],
    'clf__min_samples_split': [2, 3, 4, 5]
}

grid_searchTree = GridSearchCV(
    estimator=pipelineTree,
    param_grid=parametersTree,
    n_jobs=-1,
    verbose=1,
)

grid_searchTree.fit(X_train, Y_train)

best_parametersTree = grid_searchTree.best_estimator_.get_params()
for param_name in sorted(parametersTree.keys()):
    print(f"{param_name}: {best_parametersTree[param_name]}")


y_preds_tree_opt = grid_searchTree.predict(X_test)

print(f'MSE Tree : {mean_squared_error(Y_test, y_preds_tree_opt, squared=True)}')


Fitting 5 folds for each of 20 candidates, totalling 100 fits
clf__max_depth: 5
clf__min_samples_split: 3
MSE Tree : 76.12355396500973


Полученный результат (76.1) совсем немного превосходит линейную регрессию..

In [27]:
from sklearn.ensemble import GradientBoostingRegressor

pipelineGBR = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", GradientBoostingRegressor()),
    ]
)

parametersGBR  = {
    'clf__n_estimators': [100,150,200,50],
    'clf__learning_rate': [0.1,0.2,0.3],
}

grid_searchGBR = GridSearchCV(
    estimator=pipelineGBR,
    param_grid=parametersGBR,
    n_jobs=-1,
    verbose=1,
)

grid_searchGBR.fit(X_train, Y_train)

best_parametersGBR = grid_searchGBR.best_estimator_.get_params()
for param_name in sorted(parametersGBR.keys()):
    print(f"{param_name}: {best_parametersGBR[param_name]}")


y_preds_GBR_opt = grid_searchGBR.predict(X_test)

print(f'MSE GBR : {mean_squared_error(Y_test, y_preds_GBR_opt, squared=True)}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
clf__learning_rate: 0.2
clf__n_estimators: 50
MSE Tree : 76.05910136957364


#### Теперь попробуем повторить те же модели, но проведя Label Encoding

In [28]:

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

df = df2

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for c in df.columns:
    df[c] = le.fit_transform(df[c])

X_to_split = df.drop(['price','surge_multiplier'], inplace=False, axis=1)
Y_to_split = df['price']

X_train,  X_test, Y_train, Y_test = train_test_split(X_to_split, Y_to_split, random_state=100)

dummy1 = DummyRegressor(strategy='mean')
lr = LinearRegression()

pipelineDummy = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", dummy1),
    ]
)

pipelineLinear = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", lr),
    ]
)


pipelineDummy.fit(X_train, Y_train)
pipelineLinear.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('clf', LinearRegression())])

In [29]:
from sklearn.metrics import mean_squared_error

y_preds_dummy1 = pipelineDummy.predict(X_test)
print(f'MSE Dummy1 : {mean_squared_error(Y_test, y_preds_dummy1, squared=True)}')
y_preds_lr = pipelineLinear.predict(X_test)
print(f'MSE Linear : {mean_squared_error(Y_test, y_preds_lr, squared=True)}')

MSE Dummy1 : 443.5180559696383
MSE Linear : 232.16233744815958


In [30]:
from sklearn.neighbors import KNeighborsRegressor

nr = KNeighborsRegressor()

pipelineNR = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", nr),
    ]
)

parametersNR = {
    'clf__n_neighbors': [5,3,7,10],
    'clf__weights': ['uniform', 'distance']
}

grid_searchNR = GridSearchCV(
    estimator=pipelineNR,
    param_grid=parametersNR,
    n_jobs=-1,
    verbose=1,
)

grid_searchNR.fit(X_train, Y_train)

best_parametersNR = grid_searchNR.best_estimator_.get_params()
for param_name in sorted(parametersNR.keys()):
    print(f"{param_name}: {best_parametersNR[param_name]}")


y_preds_NR_opt = grid_searchNR.predict(X_test)

print(f'MSE Neighbors : {mean_squared_error(Y_test, y_preds_NR_opt, squared=True)}')

Fitting 5 folds for each of 8 candidates, totalling 40 fits
clf__n_neighbors: 10
clf__weights: distance
MSE Neighbors : 203.76513680686577


In [31]:
from sklearn import tree

pipelineTree = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", tree.DecisionTreeRegressor()),
    ]
)

parametersTree  = {
    'clf__max_depth': [2,3,5,7,None],
    'clf__min_samples_split': [2, 3, 4, 5]
}

grid_searchTree = GridSearchCV(
    estimator=pipelineTree,
    param_grid=parametersTree,
    n_jobs=-1,
    verbose=1,
)

grid_searchTree.fit(X_train, Y_train)

best_parametersTree = grid_searchTree.best_estimator_.get_params()
for param_name in sorted(parametersTree.keys()):
    print(f"{param_name}: {best_parametersTree[param_name]}")


y_preds_tree_opt = grid_searchTree.predict(X_test)

print(f'MSE Tree : {mean_squared_error(Y_test, y_preds_tree_opt, squared=True)}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
clf__max_depth: 7
clf__min_samples_split: 2
MSE Tree : 32.94347552031417


In [32]:
from sklearn.ensemble import GradientBoostingRegressor

pipelineGBR = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("clf", GradientBoostingRegressor()),
    ]
)

parametersGBR  = {
    'clf__n_estimators': [100,150,200,50],
    'clf__learning_rate': [0.1,0.2,0.3],
}

grid_searchGBR = GridSearchCV(
    estimator=pipelineGBR,
    param_grid=parametersGBR,
    n_jobs=-1,
    verbose=1,
)

grid_searchGBR.fit(X_train, Y_train)

best_parametersGBR = grid_searchGBR.best_estimator_.get_params()
for param_name in sorted(parametersGBR.keys()):
    print(f"{param_name}: {best_parametersGBR[param_name]}")


y_preds_GBR_opt = grid_searchGBR.predict(X_test)

print(f'MSE GBR : {mean_squared_error(Y_test, y_preds_GBR_opt, squared=True)}')

Fitting 5 folds for each of 12 candidates, totalling 60 fits
clf__learning_rate: 0.3
clf__n_estimators: 200
MSE Tree : 29.7204594247931


#### Выводы

Запишем полученные результаты в виде таблицы:

| Тип регрессора         | MSE - С категориальными значениями | MSE - Без категориальных значений |
|------------------------|------------------------------------|-----------------------------------|
| Бейслайн               | 443.5                              | 86.9                              |
| Линейный               | 232.2                              | 76.5                              |
| N ближайших соседей    | 203.8                              | 85.5                              |
| Основанный на деревьях | 32.9                               | 76.1                              |
| Градиентный бустинг    | 29.7                               | 76                                |

Видно, что при отбрасывании категориальных значений различным методам машинного обучения крайне сложно превзойти обычную линейную регрессию, следовательно, в данном случае отбрасывать подобные значения не стоит. А вот при обучении моделей с данными, включающими в себя категориальные значения, сразу два метода регрессии (основанный на деревьях и градиентный бустинг) показывают результаты, которые превышают результативность линейной регрессии почти на порядок

Оптимальные параметры для GBR- и Tree-регрессоров

GBR: learning_rate = 0.3, n_estimators = 200
Tree: max_depth = 7, min_samples_split = 2